[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ai-forever/gigachain/blob/master/docs/extras/use_cases/summarization.ipynb)

## Варианты использования

Предположим, у вас есть набор документов (PDF-файлы, страницы Notion, вопросы клиентов и т. д.), и вы хотите обобщить их содержимое.

LLM являются отличным инструментом для этого, учитывая их навыки понимания и синтеза текста.

В этом пошаговом руководстве мы рассмотрим, как выполнить обобщение документов с помощью LLM.

![Use case 1](../../static/img/summarization_use_case_1.png)

## Обзор

Основной вопрос при создании сумматора — как передать ваши документы в контекстное окно LLM. Два распространенных подхода к этому:

1.  `Stuff`: просто "сложите" все свои документы в один промпт. Это самый простой подход, который используется для документов, которые можно поместить целиком в контекст запроса.

2. `Map-reduce`: Суммаризирует каждый документ по-отдельности на шаге map, затем суммаризирует все результаты на шаге reduce для получения финального саммари.

3. `Refine`: Суммаризирует часть документов, затем добавляет новые документы к полученной суммаризации и снова суммаризирует.

4. `Conditional`: Суммаризует документы исходя из длины документы, либо методом Stuff, либо Map-reduce

![Use case 2](../../static/img/summarization_use_case_2.png)

## Quickstart

Для наглядности любой конвейер можно обернуть в один объект: `load_summarize_chain`.

Предположим, мы хотим подвести итог записи в блоге. Мы можем создать это с помощью нескольких строк кода.

Сначала установите переменные среды и установите пакеты:

In [2]:
!pip install openai tiktoken chromadb gigachain

# Set env var OPENAI_API_KEY or load from a .env file
# import dotenv

# dotenv.load_dotenv()

 При использоваии параметра `chain_type="stuff"` следует выбирать LLM с большим размером контекстного окна, например:

* 16k token OpenAI `gpt-3.5-turbo-16k` 
* 100k token Anthropic [Claude-2](https://www.anthropic.com/index/claude-2)

Также можно использовать другие типы суммаризации `chain_type="map_reduce"` или `chain_type="refine"` (подробнее смомтрите раздел [Option 3. Refine](/docs/docs/use_cases/summarization.ipynb)).

In [10]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.chat_models.gigachat import GigaChat
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

giga = GigaChat(user="<user_name>", password="<password>")
chain = load_summarize_chain(giga, chain_type="stuff")

chain.run(docs)

'В данном тексте рассматривается использование технологии машинного обучения на основе нейронных сетей для создания и управления автономными агентами. Описываются проблемы, связанные с планированием и выполнением задач на длительные периоды времени, а также с устойчивостью моделей к ошибкам. Обсуждаются преимущества использования технологии нейронных сетей для планирования и решения задач, а также ее потенциал для создания более эффективных и надежных систем искусственного интеллекта.'

## Подход 1. Stuff

When we use `load_summarize_chain` with `chain_type="stuff"`, we will use the [StuffDocumentsChain](/docs/modules/chains/document/stuff).

The chain will take a list of documents, inserts them all into a prompt, and passes that prompt to an LLM:

In [3]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

docs = loader.load()
print(stuff_chain.run(docs))

The article discusses the concept of building autonomous agents powered by large language models (LLMs). It explores the components of such agents, including planning, memory, and tool use. The article provides case studies and examples of proof-of-concept demos, highlighting the challenges and limitations of LLM-powered agents. It also includes references to related research papers and provides a citation for the article.


Great! We can see that we reproduce the earlier result using the `load_summarize_chain`.

### Go deeper

* You can easily customize the prompt. 
* You can easily try different LLMs, (e.g., [Claude](/docs/integrations/chat/anthropic)) via the `llm` parameter.

## Подход 2. Map-Reduce

Рассмотрим пример суммаризации на основании подхода map-reduce для статей, загружаемых из Википедии.

Статья из Википедии загружается целиком с помощью `WikipediaLoader`, после чего с помощью `CharacterTextSplitter` делится на части размером около 5000 символов.

In [11]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.chat_models.gigachat import GigaChat
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import CharacterTextSplitter

docs = WikipediaLoader(
    query="Винни-пух", lang="ru", load_max_docs=1, doc_content_chars_max=1000000
).load()
split_docs = CharacterTextSplitter(chunk_size=5000, chunk_overlap=500).split_documents(
    docs
)
print(f"Parts count: {len(split_docs)}")

Parts count: 12


Далее с помощью `GigaChat`` выполняется суммаризация каждой части (фаза map). В конце все суммаризированные части объединяются в одну, после чего выполняется суммарная суммаризация.

In [16]:
giga = GigaChat(profanity=False, credentials=...)
chain = load_summarize_chain(giga, chain_type="map_reduce")
res = chain.run(split_docs)

print("\n\n===")
print(res)



Винни-Пух — любимый герой детской литературы XX века, созданный Аланом Милном. Цикл о Винни-Пухе состоит из двух книг: «Винни-Пух» и «Дом на Пуховой опушке». Пересказ Бориса Заходера сохраняет языковую игру и юмор оригинала, соблюдает английскую ментальность, является популярным детским чтением и среди взрослых, особенно научной интеллигенции. Некоторые переводы были выполнены с английского оригинала, а некоторые — с сокращенной версии пересказа Заходера. Переводы Вебера и Вегушина и Лисицкой были опубликованы в 1996 году в издательстве «Моимпекс» с параллельным английским текстом, чтобы облегчить изучение языков. Однако, по мнению критиков, эти переводы не являются точными или научными, а скорее представляют собой игру с традицией или деконструкцию текста. Фильм "Винни-Пух" был создан на студии "Союзмультфильм" под руководством Федора Хитрука.

### Погружаемся глубже
 
**Кастомизация** 

* Вы можете самостоятельно настроить промпты для каждой фазы map-reduce.

**Реальные примеры использования**

* См. [this blog post](https://blog.langchain.dev/llms-to-improve-documentation/) кейс по анализу взаимодействия с пользователем (вопросы по документации LangChain)!  
* Данном посте [repo](https://github.com/mendableai/QA_clustering) представлена кластеризация как средство суммаризации.
* Это открывает третий путь, помимо подходов «stuff» или «map-reduce», который стоит рассмотреть.

![Image description](../../static/img/summarization_use_case_3.png)

## Подход 3. Refine
 
[Refine](/docs/modules/chains/document/refine) подход похож на mad-reduce:

> Цепочка refine создает ответ, перебирая входные документы и итеративно обновляя свой ответ. В цикле происходит суммаризация каждого документа с добавлением полученной на предыдущем шаге суммаризации предыдущих документов.

Вы можете легко запустить эту цепочку, использовав параметр `chain_type="refine"`.

In [22]:
chain = load_summarize_chain(llm, chain_type="refine")
chain.run(split_docs)

'The GPT-Engineer project aims to create a repository of code for specific tasks specified in natural language. It involves breaking down tasks into smaller components and seeking clarification from the user when needed. The project emphasizes the importance of implementing every detail of the architecture as code and provides guidelines for file organization, code structure, and dependencies. However, there are challenges in long-term planning and task decomposition, as well as the reliability of the natural language interface. The system has limited communication bandwidth and struggles to adjust plans when faced with unexpected errors. The reliability of model outputs is questionable, as formatting errors and rebellious behavior can occur. The conversation also includes instructions for writing the code, including laying out the core classes, functions, and methods, and providing the code in a markdown code block format. The user is reminded to ensure that the code is fully function

It's also possible to supply a prompt and return intermediate steps.

In [28]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Italian"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)

In [29]:
print(result["output_text"])

L'articolo discute il concetto di costruire agenti autonomi utilizzando LLM (large language model) come controller principale. Esplora i diversi componenti di un sistema di agenti alimentato da LLM, inclusa la pianificazione, la memoria e l'uso di strumenti. Dimostrazioni di concetto come AutoGPT mostrano la possibilità di creare agenti autonomi con LLM come controller principale. Approcci come Chain of Thought, Tree of Thoughts, LLM+P, ReAct e Reflexion consentono agli agenti autonomi di pianificare, riflettere su se stessi e migliorare iterativamente. Tuttavia, ci sono sfide legate alla lunghezza del contesto, alla pianificazione a lungo termine e alla decomposizione delle attività. Inoltre, l'affidabilità dell'interfaccia di linguaggio naturale tra LLM e componenti esterni come la memoria e gli strumenti è incerta. Nonostante ciò, l'uso di LLM come router per indirizzare le richieste ai moduli esperti più adatti è stato proposto come architettura neuro-simbolica per agenti autonomi 

In [32]:
print("\n\n".join(result["intermediate_steps"][:3]))

This article discusses the concept of building autonomous agents using LLM (large language model) as the core controller. The article explores the different components of an LLM-powered agent system, including planning, memory, and tool use. It also provides examples of proof-of-concept demos and highlights the potential of LLM as a general problem solver.

Questo articolo discute del concetto di costruire agenti autonomi utilizzando LLM (large language model) come controller principale. L'articolo esplora i diversi componenti di un sistema di agenti alimentato da LLM, inclusa la pianificazione, la memoria e l'uso degli strumenti. Vengono anche forniti esempi di dimostrazioni di proof-of-concept e si evidenzia il potenziale di LLM come risolutore generale di problemi. Inoltre, vengono presentati approcci come Chain of Thought, Tree of Thoughts, LLM+P, ReAct e Reflexion che consentono agli agenti autonomi di pianificare, riflettere su se stessi e migliorare iterativamente.

Questo artic

## Подход 4. Conditional
Данная цепочка позволяет выполнять суммаризацию методом stuff, если общий размер документов
не превышает параметр max_length (max_length указывается в количество токенов)


In [1]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.chat_models.gigachat import GigaChat

giga = GigaChat(profanity=False, model="GigaChat", user="<юзер>", password="<пароль>")

ch = load_summarize_chain(
    llm=giga, chain_type="conditional", verbose=True, max_length=1000
)

В случае ниже статья не превышает 1000 токенов, поэтому мы суммаризуем её методом Stuff

In [2]:
import re

from langchain.document_loaders import WikipediaLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs = WikipediaLoader(
    query="Улица Винни-Пуха (Варшава)",
    lang="ru",
    load_max_docs=1,
    doc_content_chars_max=1000000,
).load()
# Убираем лишние новые линии
docs = [
    Document(page_content=re.sub(r"\n\s*\n", "\n", doc.page_content)) for doc in docs
]
split_docs = RecursiveCharacterTextSplitter(
    chunk_size=5000, chunk_overlap=500
).split_documents(docs)
print(f"Parts count: {len(split_docs)}")
print(ch.run(split_docs))

Parts count: 1


> Entering new LLMChain chain...
Prompt after formatting:
Напиши краткое резюме следующего:


"Улица Винни-Пуха (польск. Ulica Kubusia Puchatka) — улица в Средместье Варшавы, которая проходит от улицы Святокрестской до улицы Варецкой.
Улица была заложена в первой половине 1950-х годов на месте руин зданий, разрушенных во время Второй мировой войны, и предназначалась для разгрузки улицы Новый Свят. Проект застройки улицы разработан архитектором Зигмунтом Стемпиньским и студентами Варшавского технологического университета.
Улица имеет длину 149 метров и ширину 23 метра, застроена четырёхэтажными жилыми зданиями, первые этажи которых предназначены для магазинов и офисов. Вдоль улицы в 1954 году были высажены два ряда лип, специально завезённых из Щецина. Неподалеку расположена станция 2-й линии варшавского метро Новы Свят — Универсытет.
Улица названа в честь персонажа известной сказки Алана Милна — плюшевого медвежонка Винни-Пуха, который изображен на табличке с названием

В случае ниже, у нас документы превышают 1000 токенов, поэтому мы используем Map-Reduce

In [3]:
docs = WikipediaLoader(
    query="Винни-Пух (мультфильм)",
    lang="ru",
    load_max_docs=1,
    doc_content_chars_max=1000000,
).load()
# Убираем лишние новые линии
docs = [
    Document(page_content=re.sub(r"\n\s*\n", "\n", doc.page_content)) for doc in docs
]
split_docs = RecursiveCharacterTextSplitter(
    chunk_size=5000, chunk_overlap=500
).split_documents(docs)
print(f"Parts count: {len(split_docs)}")
print(ch.run(split_docs))

Parts count: 2


> Entering new LLMChain chain...
Prompt after formatting:
Напиши краткое резюме следующего:


"«Ви́нни-Пух» — первый мультипликационный фильм цикла из трёх советских рисованных мультфильмов по одноимённой сказке А. Милна:
Винни-Пух (1969) (10 мин 42 с);
Винни-Пух идёт в гости (1971) (9 мин 58 с);
Винни-Пух и день забот (1972) (19 мин 25 с).
== Сюжет ==
Неугомонный медвежонок, идя по лесу и напевая песенки, встречает высокий-превысокий дуб с мёдом в улье на верхушке и с пчёлами. Сначала он пытается самовольно добраться до мёда, но ветка ломается, и он падает в репейник.
Тогда медвежонок отправился к своему лучшему другу — поросёнку Пятачку, который жил в домике с надписью «Посторонним В». Сняв колючки с носа, он попросил у него воздушный шарик, и тот дал ему синий шар. Желая замаскироваться под «тучку», Винни-Пух окунается в лужу. Возле дуба Винни-Пух надувает шар побольше, поскольку Пятачок не сумел и чуть не улетел, и воспаряет к улью.
Но Винни-Пуху, желавшему «утащит